### Multimodal RAG (PDF With Images)

## Steps into reading PDF Documents

1. PDF Document
   1. Extract Text and Images using multi-modal model (gpt-4o-mini)
   2. Convert the text into chunks by embedding using CLIP (OpenAI)
   3. Convert the images into embedding using CLIP (OpenAI)
      1. We will store the particular image in Base64 format
   4. Then we take both embeddings and store in. vector store (FAISS)
   5. We will also convert the query into an embedding using CLIP as well
   6. Then we will do a vector search (Retriever) (text+image information)
   7. Then we will convert to a specific format before passing into the LLM model (gpt-4o-mini)
   8. Now the model will receive the information
   9. And then we will receive the multi-modal answer

![Steps](content/image.png)

## Install Dependencies

In [1]:
import fitz  # PyMuPDF
from langchain_core.documents import Document
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import numpy as np
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from sklearn.metrics.pairwise import cosine_similarity
import os
import base64
import io
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

/Users/tonyribeiro/miniconda3/envs/RAG/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup

In [2]:
# Load CLIP Model
import os 
from dotenv import load_dotenv
load_dotenv()

# Set up environment
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# Init CLIP Model in HuggingFace for unified embeddings
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=True)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

## Image and Text Embedding Steps

In [3]:
# Image Embedding Steps
def embed_image(image_data):
    """Embed image using CLIP"""
    if isinstance(image_data, str): # If path
        image = Image.open(image_data).convert("RGB")
    else:
        image = image_data
        
    inputs = clip_processor(images=image,return_tensors="pt") # Returning info in tensor format
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
        
        # Normalizing embeddings to unit vector
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()
    
# Text Embedding 
def embed_text(text):
    """Embed text using CLIP"""
    inputs = clip_processor(
        text=text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=77 # CLIPs max token length
    )
    
    with torch.no_grad():
        features = clip_model.get_text_features(**inputs)
        
        # Normalize Embeddings
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()

## Reading PDF and Extracting PDF

In [5]:
# Process PDF
pdf_path = "content/article.pdf"
doc = fitz.open(pdf_path)
doc
# Storage for all documents and embeddings
all_docs = []
all_embeddings = []
image_data_store = {}

# Text Splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

for i, page in enumerate(doc):
    # Process Text
    text = page.get_text()
    if text.strip():
        # Create temporary document for splitting
        temp_doc = Document(page_content=text, metadata={"page": i, "type": "text"})
        text_chunks = splitter.split_documents([temp_doc])
        
        # Embed each chunk using CLIP
        for chunk in text_chunks:
            embedding = embed_text(chunk.page_content)
            all_embeddings.append(embedding)
            all_docs.append(chunk)
    
    # Process Image
    '''
    Three Important Actions:
        - Convert PDF Image to PIL format
        - Store as base64 for GPT 4 (which needs base64 images)
        - Create CLIP embedding for retrieval
    '''
    for img_index, img in enumerate(page.get_images(full=True)):
        try:
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            
            # Convert to PIL Image
            pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
            
            # Create unique identifier
            image_id = f"page_{i}_img_{img_index}"
            
            # Store image as base64 for later use with GPT 4o mini
            buffered = io.BytesIO()
            pil_image.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode()
            image_data_store[image_id] = img_base64
            
            # Embed Image using CLIP
            embedding = embed_image(pil_image)
            all_embeddings.append(embedding)
            
            # Create document for image
            image_doc = Document(
                page_content = f"[Image: {image_id}]",
                metadata={"page": i, "type": "image", "image_id": image_id}
            )
            all_docs.append(image_doc)
            
        except Exception as e:
            print(f"Error processing image {img_index} on page {i}: {e}")
            continue
        
doc.close()

In [6]:
all_embeddings

[array([ 3.88153568e-02,  2.73047611e-02,  2.82988092e-03, -1.55168297e-02,
         3.06821559e-02,  4.86977678e-03,  9.67360684e-04,  8.23999792e-02,
         6.37851804e-02, -2.56917402e-02,  1.50891021e-02,  1.35545107e-02,
        -2.47295573e-02,  3.42032649e-02, -2.70406529e-02,  5.63798174e-02,
         4.22723927e-02,  4.65088803e-03,  1.39950151e-02,  3.97115164e-02,
         4.28482443e-02,  3.61983664e-02,  4.49309498e-03,  3.68076796e-03,
         1.31789898e-03,  9.65760648e-03, -2.31527630e-02, -4.53921929e-02,
         1.77400478e-03, -5.54927774e-02,  1.48205487e-02, -1.49853770e-02,
        -1.83529733e-03, -3.14219370e-02,  1.34406993e-02,  2.99869739e-02,
        -3.87696712e-03, -7.99345784e-03,  5.73885906e-03, -8.21316522e-03,
         1.47494255e-02,  1.63377337e-02,  4.52326052e-03, -1.03612347e-02,
        -1.58856884e-02, -1.29548050e-02,  3.17765065e-02,  3.03454660e-02,
        -1.27789155e-02,  2.81041674e-02,  3.12426556e-02, -9.30764247e-03,
         5.9

In [7]:
all_docs

[Document(metadata={'page': 0, 'type': 'text'}, page_content='Academic Editor: Pier Luigi Mazzeo\nReceived: 26 August 2025\nRevised: 3 October 2025\nAccepted: 6 October 2025\nPublished: 8 October 2025\nCitation: Ribeiro, A.G.; Vilaça, L.;\nCosta, C.; Soares da Costa, T.;\nCarvalho, P.M. Automatic Visual\nInspection for Industrial Application.\nJ. Imaging 2025, 11, 350. https://\ndoi.org/10.3390/jimaging11100350\nCopyright: © 2025 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed under the terms and'),
 Document(metadata={'page': 0, 'type': 'text'}, page_content='This article is an open access article\ndistributed under the terms and\nconditions of the Creative Commons\nAttribution (CC BY) license\n(https://creativecommons.org/\nlicenses/by/4.0/).\nArticle\nAutomatic Visual Inspection for Industrial Application\nAntónio Gouveia Ribeiro 1,2\n, Luís Vilaça 1,2\n, Carlos Costa 3, Tiago Soares da Costa 1,4\nand Pedro Miguel Carvalho 1,5

## Creating Vector Store

In [9]:
# Create unified FAISS vector store with CLIP embeddings
embedding_array = np.array(all_embeddings)

# create custom FAISS index since we have precomputed embeddings
vector_store = FAISS.from_embeddings(
    text_embeddings=[(doc.page_content, emb) for doc, emb in zip(all_docs, embedding_array)],
    embedding=None, # We are using pre computed embeddings
    metadatas=[doc.metadata for doc in all_docs]
)

vector_store

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Calling Model (gpt-4o-mini)

In [11]:
llm = init_chat_model("openai:gpt-4o-mini")
llm

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x12bb7fa40>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12bbecdd0>, root_client=<openai.OpenAI object at 0x12adc72c0>, root_async_client=<openai.AsyncOpenAI object at 0x12bc1f380>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

## Convert Vector Store for retrieval  

In [14]:
def retrieve_multimodal(query, k=5):
    """Unified retrieval using CLIP embeddings for both text and images"""
    # Embed query using CLIP
    query_embedding = embed_text(query)
    
    # Search based on the query embeddings
    results = vector_store.similarity_search_by_vector(
        embedding=query_embedding,
        k=k 
    )

    return results

In [ ]:
def create_multimodal_message(query, retrieved_docs):
    """Create a message with both text and images for GPT-4."""
    content = []
    
    # Add the query
    content.append({
        "type": "text",
        "text": f"Question: {query}\n\nContext:\n"
    })
    
    # Separate text and image documents
    text_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "text"]
    image_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "image"]
    
    # Add text context
    if text_docs:
        text_context = "\n\n".join([
            f"[Page {doc.metadata['page']}]: {doc.page_content}"
            for doc in text_docs
        ])
        content.append({
            "type": "text",
            "text": f"Text excerpts:\n{text_context}\n"
        })
    
    # Add images
    for doc in image_docs:
        image_id = doc.metadata.get("image_id")
        if image_id and image_id in image_data_store:
            content.append({
                "type": "text",
                "text": f"\n[Image from page {doc.metadata['page']}]:\n"
            })
            content.append({
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{image_data_store[image_id]}"
                }
            })
    
    # Add instruction
    content.append({
        "type": "text",
        "text": "\n\nPlease answer the question based on the provided text and images."
    })
    
    return HumanMessage(content=content)

In [15]:
def multimodal_pdf_rag_pipeline(query):
    """Main pipeline for multimodal RAG."""
    # Retrieve relevant documents
    context_docs = retrieve_multimodal(query, k=5)
    
    # Create multimodal message
    message = create_multimodal_message(query, context_docs)
    
    # Get response from GPT-4V
    response = llm.invoke([message])
    
    # Print retrieved context info
    print(f"\nRetrieved {len(context_docs)} documents:")
    for doc in context_docs:
        doc_type = doc.metadata.get("type", "unknown")
        page = doc.metadata.get("page", "?")
        if doc_type == "text":
            preview = doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
            print(f"  - Text from page {page}: {preview}")
        else:
            print(f"  - Image from page {page}")
    print("\n")
    
    return response.content

In [17]:
if __name__ == "__main__":
    # Example queries
    queries = [
        "What does the table on page 23 show about LwF along the time?",
        "Summarize the main findings from the document",
        "What visual elements are present in the document?"
    ]
    
    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)
        answer = multimodal_pdf_rag_pipeline(query)
        print(f"Answer: {answer}")
        print("=" * 70)


Query: What does the table on page 23 show about LwF along the time?
--------------------------------------------------

Retrieved 5 documents:
  - Text from page 24: from a publicly funded project.
  - Text from page 26: J. Imaging 2025, 11, 350
27 of 30
(c)
(d)
(e)
(f)
(g)
(h)
Figure A1. Cont.
  - Text from page 22: 1
63.0
83.0
61.2
81.4
2
61.3
79.0
57.4
80.2
3
53.0
80.8
52.1
76.1
4
60.2
77.1
31.1
71.5
5
60.2
78.3
...
  - Text from page 15: 0.209
0.919
0.940
0.945
0.759
23.1
70/10/20
8
0.193
0.380
0.233
0.914
0.916
0.921
0.738
22.6
BDDIE
Y...
  - Text from page 17: 0.027
0.027
0.2542
0.9701
0.6109
0.4767
68.42
50
16
1.1167
0.011
0.011
0.4827
0.8678
0.6731
0.5215
6...


Answer: The excerpts provided do not include specific information about the content of the table on page 23, as the actual data or details from that table are not mentioned. To accurately answer the question regarding what the table shows about LwF (Learning without Forgetting) over time, I would need direct informati